In [0]:
%load_ext autoreload
%autoreload 2
# Enables autoreload; learn more at https://docs.databricks.com/en/files/workspace-modules.html#autoreload-for-python-modules
# To disable autoreload; run %autoreload 0

In [0]:
from validation_rules import not_null_rule, date_format_rule, concat_errors
from pyspark.sql.functions import col
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

df = spark.read.table("my_database.my_schema.cln_ar_receipt_cleanse")

invalid_customer_name = not_null_rule("customer_name")
invalid_date_format = date_format_rule("date")
invalid_date_null = not_null_rule("date")

# Combine errors
Invalid_summary = concat_errors(invalid_customer_name, invalid_date_format,invalid_date_null)


In [0]:
# Add error column(s) at the end of DataFrame
df_validated = (df.withColumn("invalid_customer_name", invalid_customer_name).withColumn("invalid_date_format", invalid_date_format).withColumn("invalid_date_null", invalid_date_null).withColumn("Invalid_summary", Invalid_summary))


#df_validated = df_validated.withColumn("invalid_date_format", invalid_date_format)
#df_validated = df_validated.withColumn("invalid_date_null", invalid_date_null)
#df_validated = df_validated.withColumn("Invalid_summary", Invalid_summary)

# Split valid/invalid
df_valid = df_validated.filter(col("Invalid_summary") == "")
df_invalid = df_validated.filter(col("Invalid_summary") != "")

df_invalid.show()

In [0]:
# Write outputs (dummy table names)
df_valid.write.mode("overwrite").saveAsTable("my_database.my_schema.vld_ar_receipt_valid")
df_invalid.write.mode("overwrite").saveAsTable("my_database.my_schema.vld_ar_receipt_invalid")